Apéndice --- Análisis del desempeño del modelo en R
===

* *30 min* | Ultima modificación: Junio 22, 2019

---

## Partición de los datos

Los resultados de la evaluación de los modelos es dependiente de los datos usados. En la práctica, los datos se suelen partir en tres conjuntos, tal como muestra la gráfica de abajo:


* Conjunto de calibración de parámetros del modelo.


* Conjunto de prueba, usado comunmente para determinar la complejidad del modelo o el valor óptimo de alguno de sus parámetros.


* Conjunto de pronóstico, en el que se intenta reproducir el comportamiento del modelo en productivo.

![assets/data-partition.jpg](assets/data-partition.jpg)

En la figura anterior, los datos se dividen secuencialmente, pero podría construirse cada conjunto aletaoriamente. 

Si se tiene en cuenta que hay muchas particiones aleatorias posibles, una mejor estimación de la matriz de confusión (o cualquier otro estadístico que se calcule para un conjunto de datos) podría ser tomando los valores esperados de cada métrica. Es decir, si se repite el experimento $N$ veces, se tendrían $N$ valores posibles para cada uno de los elementos de la matriz de confusión y por lo tanto se podría tener su valor medio. Esta sería una métrica mucho más apropiada.

A continuación se describen varios mecanismos para gestionar la forma en que los datos son particionados.

In [1]:
## 
## El paquete caret contiene las funciones para 
## partir los datos y entrenar los modelos
## 
library(caret)

Loading required package: lattice
Loading required package: ggplot2


In [2]:
##
## Se crea una muetra de números enteros
##
data <- 1:20

In [3]:
##
## Crea muestras sin repetir elementos en cada grupo 
##
folds = createDataPartition(y = data,    # datos 
                            times = 7,   # número de particiones 
                            p = 0.4)     # porcentaje de los datos a usar en el entrenamiento
str(folds)

List of 7
 $ Resample1: int [1:8] 3 4 6 9 11 12 16 19
 $ Resample2: int [1:8] 3 5 7 10 12 14 16 18
 $ Resample3: int [1:8] 1 5 6 9 11 12 16 17
 $ Resample4: int [1:8] 1 5 9 10 11 12 18 20
 $ Resample5: int [1:8] 1 5 6 7 12 15 17 20
 $ Resample6: int [1:8] 4 5 8 9 14 15 17 20
 $ Resample7: int [1:8] 1 5 6 7 12 15 17 18


## K-fold crossvalidation

En este método, el conjunto de datos para entrenamiento (ajuste + prueba) es dividido en $K$ grupos. Este es un proceso iterativo que opera de la siguiente forma (véase la figura de abajo). 


* Se toma el grupo 1 como conjunto de datos de prueba (grupo rojo) y se entrena el modelo con los grupos restantes {2, ..., K} (grupo negro).


* Se toma el grupo 2 como conjunto de datos de prueba (grupo rojo) y se entrena el modelo con los grupos restantes {1, 3, ..., K} (grupo negro).


* Se continua de esta forma hasta que se usa el grupo K para prueba, mientras que se usan los grupos 1 hasta K-1 para entrenamiento.



![assets/k-fold-crossval.jpg](assets/k-fold-crossval.jpg)

De esta forma, se tienen K valores posibles para el estadístico de interés. Usualmente se reporta su valor promedio.

Note que una mejor opción sería distribuir los datos en cada grupo de forma aleatoria.

In [4]:
##
## Crea k grupos sin repetir elementos
##
folds <- createFolds(data,   # datos 
                     k = 7)  # cantidad de grupos
str(folds)

List of 7
 $ Fold1: int [1:3] 4 7 12
 $ Fold2: int [1:3] 9 14 18
 $ Fold3: int [1:2] 10 15
 $ Fold4: int [1:3] 2 11 17
 $ Fold5: int [1:3] 5 13 20
 $ Fold6: int [1:3] 3 6 8
 $ Fold7: int [1:3] 1 16 19


## Bootstrap

El bootstrap se basa en el remuestreo de los datos para poder obtener una muestra del estadístico que se está calculado (por ejemplo, la cantidad de falsos positivos FP). 

Supoga que tiene una muestra de ocho ejemplos:


$$\{x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8\}$$

Una muestra bootstap se obtiene de la muestra original, seleccionando ocho elementos de forma aleatoria. Por ejemplo, una muestra bootstrap podría ser:

$$\{x_1, x_2, x_2, x_2, x_4, x_1, x_7, x_7\}$$

Nóte que los elementos pueden repetirse. Sobre cada muestra bootstrap obtenida, se realiza el proceso de cómputo y se obtiene el estadístico de interés. Si este procedimiento se repite 500 veces para calcular la cantidad de FP, se tenddrían 500 valores posibles de FP. Una estimación mucho mejor del valor de FP, sería calcular su valor promedio a partir de la muestra de 500 valores. Más aún, podría calcularse el histograma o la distribución de probabilidades de FP, lo cual es mucho más informativo sobre el desempeño del modelo.

In [5]:
##
## Bootstrap del mismo tamaño de 
## la muestra original
##
folds <- createResample(data,   # datos 
                        times = 7)  # cantidad de grupos
str(folds)

List of 7
 $ Resample1: int [1:20] 2 2 3 5 5 6 6 6 6 7 ...
 $ Resample2: int [1:20] 1 1 3 3 4 5 7 11 11 11 ...
 $ Resample3: int [1:20] 1 2 2 4 4 6 6 8 8 9 ...
 $ Resample4: int [1:20] 1 1 3 3 4 5 5 7 8 9 ...
 $ Resample5: int [1:20] 1 1 1 3 3 4 5 7 9 9 ...
 $ Resample6: int [1:20] 1 2 3 3 4 4 5 5 6 6 ...
 $ Resample7: int [1:20] 1 3 5 6 7 9 9 10 10 11 ...


## Leave-one-out

Este es el K-fold con K=1.

![assets/leave-one-out.jpg](assets/leave-one-out.jpg)

## Uso en el entrenamiento de un modelo

In [6]:
##
## Se carga la librería
##
library(C50)

##
## Lectura de los datos
##
data <- read.csv("data/credit.csv")
data$default <- factor(data$default, labels=c("No", "Yes"))

##
## se crean 10 grupos
##
folds <- createFolds(data$default, k = 10)
str(folds)

List of 10
 $ Fold01: int [1:100] 5 11 21 25 27 29 38 45 67 118 ...
 $ Fold02: int [1:100] 2 4 12 39 55 59 61 77 104 107 ...
 $ Fold03: int [1:100] 8 15 35 43 56 70 76 79 84 94 ...
 $ Fold04: int [1:100] 1 20 24 31 34 36 42 64 68 83 ...
 $ Fold05: int [1:100] 10 14 16 22 26 40 63 69 78 82 ...
 $ Fold06: int [1:100] 9 13 37 49 51 52 62 65 71 72 ...
 $ Fold07: int [1:100] 17 19 44 93 95 97 99 102 117 134 ...
 $ Fold08: int [1:100] 7 18 28 41 50 60 75 80 81 86 ...
 $ Fold09: int [1:100] 3 23 32 47 48 53 57 66 74 101 ...
 $ Fold10: int [1:100] 6 30 33 46 54 58 87 89 90 106 ...


In [7]:
##
## Se crea una función que aplica el método 
## a los datos
##
library(irr)

f <- function(fold) {
    
    ## elimina el grupo del patron de entrenamiento
    X_train <- data[-fold, ]
    y_train <- data$default[-fold]
    
    ## usa el grupo como conjunto de prueba
    X_test      <- data[fold, ]
    y_test_true <- data$default[fold]
    
    ## entrena el modelo y pronostica
    clf <- C5.0(default ~ ., data = X_train)
    y_test_pred <- predict(clf, X_test)

    
    ## calcula una métrica de error
    kappa <- kappa2(data.frame(y_test_true, y_test_pred))$value
    
    ## retorna el valor calculado
    return(kappa)
  }


##
## Esta variable guarda los resultados para cada uno 
## de los 10 grupos.
##
cv_results <- lapply(folds, f)

# Se calcula kappa para cada uno de los grupos
str(cv_results)

Loading required package: lpSolve


List of 10
 $ Fold01: num 0.194
 $ Fold02: num 0.495
 $ Fold03: num 0.206
 $ Fold04: num 0.417
 $ Fold05: num 0.351
 $ Fold06: num 0.297
 $ Fold07: num 0.175
 $ Fold08: num 0.193
 $ Fold09: num 0.175
 $ Fold10: num 0.459


In [8]:
##
## Se reporta el promedio de kappa 
##
mean(unlist(cv_results))

[1] 0.2962101

## Afinación de los parámetros de modelos

Se procede a computar el modelo óptimo para el problema del riesgo de crédito.

In [9]:
##
## Se obtienen los parámetros ajustables del modelo
## considerado
##
modelLookup("C5.0")

model,parameter,label,forReg,forClass,probModel
C5.0,trials,# Boosting Iterations,FALSE,TRUE,TRUE
C5.0,model,Model Type,FALSE,TRUE,TRUE
C5.0,winnow,Winnow,FALSE,TRUE,TRUE


In [10]:
##
## La función `train` permite construir muchos modelos
## que difieren en el conjunto de parámetros ajustables
## requeridos para su estimación. Se usan los valores
## por defecto que tiene `train`.
## Note que este paso requiere mucho tiempo de cómputo.
##
set.seed(300)
m <- train(default ~ ., data = data, method = "C5.0")
m

C5.0 

1000 samples
  20 predictor
   2 classes: 'No', 'Yes' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 1000, 1000, 1000, 1000, 1000, 1000, ... 
Resampling results across tuning parameters:

  model  winnow  trials  Accuracy   Kappa    
  rules  FALSE    1      0.6923931  0.2736692
  rules  FALSE   10      0.7221394  0.3408652
  rules  FALSE   20      0.7347490  0.3622943
  rules   TRUE    1      0.6913853  0.2693031
  rules   TRUE   10      0.7156087  0.3252182
  rules   TRUE   20      0.7264898  0.3438105
  tree   FALSE    1      0.6922255  0.2564458
  tree   FALSE   10      0.7308796  0.3120268
  tree   FALSE   20      0.7327493  0.3166876
  tree    TRUE    1      0.6871812  0.2478289
  tree    TRUE   10      0.7338952  0.3250631
  tree    TRUE   20      0.7353105  0.3266443

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were trials = 20, model = tree and winnow
 = TRUE.

Note que en la tabla anterior, el valor del estadístico de ajuste para cada corrida es obtenido sobre 25 réplicas bootstrap para cada conjunto posible de parámetros. Los valores posibles que puede tomar cada parámetro son:


* `model` = {rules, tree}


* `winnow` = {TRUE, FALSE}


* `trials` = {1, 10, 20}

Entonces, 2 x 2 x 3 = 12.

Este proceso es equivalente a realizar un diseño de experimentos para obtener la mejor combinación de parámetros.

In [11]:
##
## Después de identificar el mejor modelo, `train()` 
## usa el mejor conjunto de parámetros para construir 
## un modelo estimado sobre todo el conjunto de datos.
##
## Mejor modelo:
##
m$finalModel


Call:
(function (x, y, trials = 1, rules = FALSE, weights = NULL, control
 "winnow", "noGlobalPruning", "CF", "minCases", "fuzzyThreshold",
 "sample", "earlyStopping", "label", "seed")))

Classification Tree
Number of samples: 1000 
Number of predictors: 48 

Number of boosting iterations: 20 
Average tree size: 45.1 

Non-standard options: attempt to group attributes, winnowing


In [13]:
##
## Pronóstico
##
p <- predict(m, data)
table(p, data$default)

     
p      No Yes
  No  676  79
  Yes  24 221

In [14]:
##
## Probabilidades del pronóstico para la muestra de entrenamiento
##
head(predict(m, data, type = "prob"))

No,Yes
0.8720819,0.12791809
0.3284062,0.67159380
1.0000000,0.00000000
0.7563177,0.24368229
0.4531721,0.54682785
0.9085110,0.09148904


In [15]:
##
## A continuación se ejemplifica como parametrizar
## la búsqueda realizada por train.
##
ctrl <- trainControl(method = "cv",               # cross validation
                     number = 10,                 # número de grupos
                     selectionFunction = "oneSE") # 

In [16]:
##
## Se crea una malla de valores posibles
## para los parámetros a optimizar
##
grid <- expand.grid(.model = "tree",
                    .trials = c(1, 5, 10, 15, 20, 25, 30, 35),
                    .winnow = "FALSE")
grid

.model,.trials,.winnow
tree,1,FALSE
tree,5,FALSE
tree,10,FALSE
tree,15,FALSE
tree,20,FALSE
tree,25,FALSE
tree,30,FALSE
tree,35,FALSE


In [18]:
##
## Se llama a train() con los valores 
## suministrados para la búsqueda.
##
set.seed(300)
m <- train(default ~ .,         # todas las variables
           data      = data,    # dataset
           method    = "C5.0",  #
           metric    = "Kappa", #
           trControl = ctrl,    #
           tuneGrid  = grid)    #
m

Warning message in Ops.factor(x$winnow):
“‘!’ not meaningful for factors”

C5.0 

1000 samples
  20 predictor
   2 classes: 'No', 'Yes' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results across tuning parameters:

  trials  Accuracy  Kappa    
   1      0.714     0.2918834
   5      0.735     0.3401066
  10      0.747     0.3504832
  15      0.749     0.3642719
  20      0.755     0.3641425
  25      0.749     0.3521465
  30      0.744     0.3445306
  35      0.745     0.3517542

Tuning parameter 'model' was held constant at a value of tree
Tuning
 parameter 'winnow' was held constant at a value of FALSE
Kappa was used to select the optimal model using  the one SE rule.
The final values used for the model were trials = 5, model = tree and winnow
 = FALSE.

**Ejercicio.--** Para el clasificador kNN usando en la predicción del tumor de cancer de mama, compute y grafique el error de validación cruzada para distintos valores de K. Según los resultados obtenidos, cuál es el valor recomendado de K?